In [2]:
# Import modules
import numpy as np
import numpy.linalg as LA

# Import functions
from assembly_A_local_global_matrices import *
from assembly_K_matrices import *
from assembly_B_matrices import *
from assembly_f_vectors import *
from assembly_u_solution import *
from RegularSudomainsMesh import RegularSubdomainsMesh
from utils import *

%load_ext autoreload
%autoreload 2

In [3]:
# Import data
d_dat = np.genfromtxt('data/d.dat')
fP_dat = np.genfromtxt('data/fP.dat')
fr_dat = np.genfromtxt('data/fr.dat')
Ks = np.genfromtxt('data/localK.dat')
solution = np.genfromtxt('data/solution.dat')

In [4]:
# Initial data
# Number of subdomains
Nsub_x = 4
Nsub_y = 3

# Number of remaining nodes in each subdomain
Nr_x = 4
Nr_y = 3

# Local remaining and primal indices
rs = np.array([1, 2, 4, 5, 6, 7, 9, 10])
qs = np.array([0, 3, 8, 11])
qs_left_bound = np.array([0, 8])
qs_right = np.array([3, 11])

In [5]:
# Create mesh
mesh = RegularSubdomainsMesh(Nsub_x, Nsub_y, Nr_x, Nr_y)

In [6]:
# Transformation matrices A
# Primal nodes local-global transformation matrices
APq = create_APq_matrices(mesh)

# Remaining nodes local-global transformation matrices
ARr = create_ARr_matrices(mesh)

# Dirichlet nodes local-global transformation matrices
ADd = create_ADq_matrices(mesh)

In [7]:
# Stiffness matrices K
KRR = assembly_KRR_matrix(Ks, ARr, rs, mesh)
KPP = assembly_KPP_matrix(Ks, APq, qs, qs_right, mesh)
KRP, Krqs_list = assembly_KRP_matrix(Ks, APq, ARr, qs, qs_right, rs, mesh)
KPR = KRP.T
Kqrs_list = [Krqs.T for Krqs in Krqs_list]
KPD = assembly_KPD_matrix(Ks, APq, ADd, qs_left_bound, qs_right, mesh)
KRD = assembly_KRD_matrix(Ks, ARr, ADd, qs_left_bound, rs, mesh)

In [8]:
# Assembly B matrices
BlambdaR, BRs_list = assembly_BR_matrix(mesh, ARr)

# Dirichlet boundary conditions
# Left wall remaining
BlambdaR, BRs_list = assembly_Dirichlet_BR_matrix(mesh, ARr, BlambdaR, BRs_list)

In [9]:
# Assembly d vector
d = np.zeros(mesh.Nlambda)
d[mesh.NlambdaR:] = d_dat[np.arange(len(d_dat)) % (mesh.Nr_y - 1) != 0]

In [10]:
# Assembly f vectors
# fP
fP, fD = assembly_fP_fD_vectors(mesh, fP_dat)

# fR
fR = assembly_fR_vector(mesh, fr_dat)

In [11]:
# Assembly uD vector
uD = d_dat[np.arange(len(d_dat)) % (mesh.Nr_y - 1) == 0]

In [12]:
# Matrices pre computation

In [13]:
# Matrices pre computation
KRR_inv = np.linalg.inv(KRR)

SPP = KPP - KPR @ KRR_inv @ KRP
SPP_inv = np.linalg.inv(SPP)

fPH = fP - KPD @ uD
fRH = fR - KRD @ uD

IR = np.eye(mesh.NR) # RxR identity matrix

dH = d - BlambdaR @ KRR_inv @ ((IR + KRP @ SPP_inv @ KPR @ KRR_inv) @ fRH - KRP @ SPP_inv @ fPH)
F = -BlambdaR @ KRR_inv @ (KRP @ SPP_inv @ KPR @ KRR_inv + IR) @ BlambdaR.T

In [14]:
z = np.array([0.11097952, 0.76367948, 0.3643149 , 0.43658249, 0.54056894,
       0.62960657, 0.46773485, 0.57270768, 0.02533156, 0.96696452,
       0.33969156, 0.23758129, 0.2509545 , 0.84673038, 0.435943  ,
       0.19422253, 0.61608355, 0.24816213, 0.08060656, 0.04745991,
       0.03908529, 0.70291525, 0.35247435, 0.63742649, 0.21508681,
       0.77903499, 0.27058676, 0.29763978])

In [15]:
np.shape(KRR_inv)

(96, 96)

In [16]:
from scipy.linalg import solve_triangular

def solve_cholesky(L, b):
    """
    Solves the system of linear equations Ax = b using the Cholesky decomposition,
    where A is a symmetric positive definite matrix and L is its Cholesky factorization.
    This function uses triangular solvers to perform forward and backward substitution.
    
    Parameters:
    L (ndarray): The lower triangular Cholesky factor of A
    b (ndarray): The right-hand side vector of the equation Ax = b
    
    Returns:
    x (ndarray): The solution vector x of Ax = b
    """
    # Solve Ly = b using forward substitution
    y = solve_triangular(L, b, lower=True)
    
    # Solve L.T x = y using backward substitution
    x = solve_triangular(L.T, y, lower=False)
    
    return x


c:\Users\34683\.conda\envs\ada\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [19]:
# Matrix vector multiplication
# xr = KPR @ KRR_inv @ BlambdaR.T @ z
x = np.zeros(np.shape(KPR)[0])

Krrs = Ks[rs][:, rs]
Kqrs = Ks[qs][:, rs]
Krrs_inv = LA.inv(Krrs)
LA_Krrs = LA.cholesky(Krrs)
LA_SPP = LA.cholesky(SPP)
Krrs_inv = LA.inv(Krrs)

# In parallel
for (Apqs, Kqrs, Brs) in zip (APq, Kqrs_list, BRs_list):
    x += Apqs @ Kqrs @ Krrs_inv @ Brs.T @ z

alpha = solve_cholesky(LA_SPP, x)

a = np.zeros(np.shape(BlambdaR)[0])
ar = BlambdaR @ KRR_inv @ KRP @ alpha

# Sync required!
for (BRs, Kqrs, Apqs) in zip (BRs_list, Kqrs_list, APq):
    a += BRs @ Krrs_inv @ Kqrs.T @ Apqs.T @ alpha

b = np.zeros(np.shape(BlambdaR)[0])
br = BlambdaR @ KRR_inv @ BlambdaR.T @ z

for BRs in BRs_list:
    b += BRs @ Krrs_inv @ BRs.T @ z


In [20]:
err = ar - a
err[err < 1e-10] = 0
err

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])